# High-Order Procedures
Abbiamo visto come le procedure siano in effetti delle **ASTRAZIONI** che ci permettono di descrivere delle operazioni composte in termini di numeri che sono indipendenti dai particolari valori che quei numeri possono assumere. Per esempio, quando definiamo:

In [ ]:
def Cubo(x):
    return x*x*x   # equivalente a x**3

non stiamo specificando il cubo di un numero particolare, ma stiamo specificando un metodo per trovare il cubo di un numero qualsiasi.

Un linguaggio di programmazione per essere utile deve fornire un metodo per definire astrazioni di questo tipo, ovvero operazioni composte da operazioni primitive del linguaggio, in modo da poter ragionare solo nei termini più astratti.

Tuttavia, un linguaggio di programmazione non deve essere limitato ad accettare come parametri solo dati numerici, ma dovrebbe essere in grado di accettare come parametri delle procedure stesse, ed eventualmente restituire delle procedure invece che dei semplici numeri.

In questo notebook, vedremo dei semplici esempi in cui le procedure che definiamo nel linguaggio possono essere usate come parametri di altre funzioni composte, oppure come valore di ritorno di una procedura.

## Procedure passate come argomenti.
Si considerino le tre procedure seguenti. La prima calcola la somma di interi da $a$ a $b$:

In [ ]:
def SommaInteri(a, b):
    if a > b:
        return 0
    else:
        return a + SommaInteri(a+1, b)
    
SommaInteri(1,5)

La seconda procedura calcola la somma dei cubi all'interno di un intervallo dato:

In [ ]:
def SommaCubi(a, b):
    if a > b:
        return 0
    else:
        return Cubo(a) + SommaCubi(a+1, b)
    
SommaCubi(1,5)

La terza procedura calcola la somma di una sequenza di termini nella serie:

$$
    \frac{1}{1 \cdot 3} + \frac{1}{5 \cdot 7} + \frac{1}{9 \cdot 11} + ...
$$

che converge a $\frac{\pi}{8}$ (anche se molto lentamente):

In [ ]:
def SommaPI(a, b):
    if a > b:
        return 0
    else:
        return (1/(a*(a+2))) + SommaPI(a+4, b)

Queste tre procedure hanno uno schema di calcolo comune. Sono per lo più identiche, e si differenziano solo nel nome della procedura, la funzione di $a$ che viene usata per calcolare il valore che viene sommato, e la funzione che definisce il prossimo valore di $a$ che deve essere usato.

Potremmo generare ognuna di queste tre procedure, partendo dallo schema seguente:

```
def <NomeProcedura>(a, b):
    if a > b:
        return 0
    else:
        return <F>(a) + <NomeProcedura>(<Next>(a), b)
```

La presenza di uno schema comune alla tre procedure rende evidente che si sia un'*astrazione* che aspetta solo di emergere. In pratica, i **matematici** hanno identificato questo schema tanti anni fa, identificando il concetto astratto di sommatoria di funzioni e hanno inventato la *notazione sigma* seguente:

$$
\sum_{n=a}^b f(n) = f(a) + \dots f(b)
$$

per esprimere in modo conciso questo concetto astratto.

In maniera analoga, come programmatori, vorremo avere un linguaggio che ci permetta di scrivere una procedura che rappresenti il concetto di sommatoria di funzioni, piuttosto che scrivere una procedura che calcoli solo una particolare operazione composta.

Questo può essere fatto in maniera abbastanza diretta in Python, nel modo seguente:

In [ ]:
def Sommatoria(F, a, Next, b):
    if a > b:
        return 0
    else:
        return F(a) + Sommatoria(F, Next(a), Next, b)

Si noti che in questo caso, $a$ e $b$ sono gli estremi dell'intervallo della sommatoria, mentre `F` e `Next` sono due procedure non ancora specificate.

Possiamo usare la procedura `Sommatoria` nel modo seguente.

La prima procedura vista può essere definita come:

In [ ]:
def Inc(x):
    return x+1

def SumCubi(a, b):    
    return Sommatoria(Cubo, a, Inc, b)

SumCubi(1, 5)

Utilizzando la funzione identità per calcolare al posto della funzione `Cubo`, possiamo definire anche la prima funzione vista, ovvero la funzione `SommaInteri`, nel modo seguente:

In [ ]:
def Ident(x):
    return x

def SumInteri(a, b):
    return Sommatoria(Ident, a, Inc, b)

SumInteri(1,5)

Infine, facendo attenzione, possiamo definire anche la terza procedura, quella per calcolare $\frac{\pi}{8}$, utilizzando sempre la funzione high-order `Sommatoria`:

In [ ]:
def SommaPI(a, b):
    def PIF(x):
        return 1/(x*(x+2))
    def PINext(x):
        return x+4
    
    return Sommatoria(PIF, a, PINext, b)

print(8*SommaPI(1,1000)) # Moltiplichiamo per 8, per ottenere $\pi$

### Calcolo di integrali definiti
Ora che abbiamo la funzione `Sommatoria`, possiamo usarla per formulare anche altri concetti. Per esempio, l'integrale definito di una funzione $f$ tra i limiti $a$ e $b$, può essere approssimato numericamente usando la formula:

$$
    \int_a^b = [f(a+dx/2) + f(a+dx+dx/2) + f(1+ 2dx + dx/2)+ ...]dx
$$

per valori piccoli di $dx$. Questo può essere implementato direttamente in una procedura come segue:

In [ ]:
def Integrale(F, a, b, dx):
    def AddDx(x):
        return x + dx
    return dx*Sommatoria(F, a+dx/2, AddDx, b)

Integrale(Cubo, 0, 1, 0.05)

(Si tenga presente che il valore esatto dell'integrale è $\frac{1}{4}$).

### Definizione di procedure <`lambda`>
Negli esempi precedenti abbiamo definito alcune procedure molto semplici, il cui unico scopo era quello di usarle come parametri alla chiamata di altre procedure, come ad esempio le procedure `AddDx` e `Ident`.

In molti linguaggi, quando si devono definire delle procedure per un uso "locale", si usano quelle che in gergo vengono chiamate le "*lambda functions*". In pratica, si introduce nella **sintassi** del linguaggio, la possibilità di definire delle procedure senza specificarne direttamente il nome, ed usandole direttamente come parametri di chiamate ad altre procedure. La sintassi in Python è la seguente:

```
lambda x: <corpo della procedura, dove viene usata x>
```

La prima è la parola chiave `lambda`, che è una parola chiave del linguaggio, che viene seguita da una lista di parametri formali (di solito se ne usa solo uno), poi dopo i due punti segue il corpo della procedura.

Per esempio la procedura `Integrale` definita sopra potrebbe essere usata nel modo seguente, per calcolare l'integrale di $x^4$

In [ ]:
Integrale(lambda x: x**4, 0, 1, 0.05)

In [ ]:
Integrale(lambda x: x**3, 0, 1, 0.05)

Si noti come si è definita una piccola procedura, senza darle un nome, ed usandola solo come parametro alla chiamata di un'altra procedura.

In maniera analoga, la funzione `SommaPI` potrebbe essere definita direttamente nel modo seguente:

In [ ]:
def SommaPILambda(a, b):
    return Sommatoria(lambda x: 1.0/(x*(x+2)), a, lambda x: x+4, b)

print(8*SommaPILambda(1,1000))

### Procedure come oggetti restituiti da una procedura
Le procedure, sia quelle specificate con un nome, sia quelle definite tramite la parola chiave `lambda`, posso essere utilizzate come valore di ritorno di una procedura.

Consideriamo l'esempio seguente. Si vuole scrivere una procedura che implementi la funzione composta

$$
    f(x) = 2\,g(x)
$$

Senza specificare quale sia la funzione $g(x)$, che deve essere un parametro di input della nostra funzione. Usando le `lambda` possiamo scrivere la seguente procedura:

In [ ]:
def FunFactory(G):
    return lambda x: 2*G(x)

In questo caso, abbiamo scritto una procedura chiamata `FunFactory` che prende in input una procedura `G`, e restituisce in output una procedura `lambda` che calcola la funzione `2*G(x)`.

Questa funzione **HIGH-ORDER**, può essere utilizzata nel modo seguente, per calcolare $f_1(x)= 2\, x^2$ e $f_2(x)=2\, x^3$:

In [ ]:
F1 = FunFactory(lambda x: x**2)
F2 = FunFactory(lambda x: x**3)

print(F1(3), F2(3))

**NOTA:** La possibilità di definire procedure/funzioni HIGH-ORDER, che prendono in input delle procedure e/o restituiscono in output altre procedure, è una caratteristica molto importante di un linguaggio di programmazione, che permette di introdurre in maniera astratta, ma elegante, procedure che possono effettuare dei calcoli anche complessi.